In [2]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator


2024-03-30 09:35:28.686017: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-30 09:35:29.330855: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-30 09:35:29.330906: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-30 09:35:29.462539: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-30 09:35:29.707169: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# Define data paths
data_path = r'final data/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)'
train_data_dir = os.path.join(data_path, 'train')
valid_data_dir = os.path.join(data_path, 'valid')

In [4]:
# Define data generators
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

In [13]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


valid_generator = valid_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(128, 128),
    batch_size=64,
    class_mode='categorical')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(128, 128),
    batch_size=64,
    class_mode='categorical'
)


Found 17572 images belonging to 38 classes.
Found 70295 images belonging to 38 classes.


In [14]:
# Define ResNet-based model using TensorFlow/Keras
def create_resnet_model():
    base_model = tf.keras.applications.ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(128, 128, 3)
    )
    for layer in base_model.layers:
        layer.trainable = False  # Freeze base layers
    
    x = layers.Flatten()(base_model.output)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(len(train_generator.class_indices), activation='softmax')(x)
    
    model = models.Model(inputs=base_model.input, outputs=outputs)
    return model


In [15]:
# Initialize the ResNet-based model
model_resnet_tf = create_resnet_model()

In [16]:
# Compile the model
model_resnet_tf.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history_resnet_tf = model_resnet_tf.fit(train_generator, epochs=10, validation_data=valid_generator)

Epoch 1/10


2024-03-30 09:42:29.306821: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type CPU is enabled.


1099/1099 [==============================] - 571s 518ms/step - loss: 3.6001 - accuracy: 0.0378 - val_loss: 3.4893 - val_accuracy: 0.0479
Epoch 2/10
 741/1099 [===================>..........] - ETA: 2:32 - loss: 3.5334 - accuracy: 0.0493

In [ ]:
history_resnet_tf = model_resnet_tf.fit(train_generator, epochs=15, validation_data=valid_generator)

In [17]:

# Plot training history
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

# Plot training history for TensorFlow ResNet model
plot_history(history_resnet_tf)

NameError: name 'history_resnet_tf' is not defined

In [15]:

# Save the trained ResNet model
model_resnet_tf.save('plantDiseaseModelResNetTF.h5')

/home/ua4c0cc60a319152129bfdb48913aa66/.local/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
